## Screening Task 3

In this notebook, we will try to solve Problem 3 of the screening task.

In [1]:
from docplex.mp.model import Model
from ilp_to_qubo import *

First, we create a DOcplex model for one instance of the bin packing problem.

In [2]:
bp_model_1 = Model(name='bin_packing')

# Size of the problem
n = 2

# Preparing a list of decision variables
x = []
for i in range(n):
    for j in range(n):
        x.append(bp_model_1.binary_var(name='x_'+str(i)+'_'+str(j)))

y = []
for j in range(n):
    y.append(bp_model_1.binary_var(name='y_'+str(j)))


# Weight function as input
w = [2,5]

# Capacity of the bins as input
c = 7

# Adding the constraints to the model
for j in range(n):
    sum = 0
    for i in range(n):
        sum += w[i]*x[(n*i) + j]
    bp_model_1.add_constraint(sum <= y[j]*c)

for i in range(n):
    sum = 0
    for j in range(n):
        sum += x[(n*i) + j]
    bp_model_1.add_constraint(sum == 1)

# Addiing the objective function
sum = 0
for j in range(n):
    sum += y[j]
bp_model_1.minimize(sum)

In [3]:
print(bp_model_1.lp_string)

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: bin_packing

Minimize
 obj: y_0 + y_1
Subject To
 c1: 2 x_0_0 + 5 x_1_0 - 7 y_0 <= 0
 c2: 2 x_0_1 + 5 x_1_1 - 7 y_1 <= 0
 c3: x_0_0 + x_0_1 = 1
 c4: x_1_0 + x_1_1 = 1

Bounds
 0 <= x_0_0 <= 1
 0 <= x_0_1 <= 1
 0 <= x_1_0 <= 1
 0 <= x_1_1 <= 1
 0 <= y_0 <= 1
 0 <= y_1 <= 1

Binaries
 x_0_0 x_0_1 x_1_0 x_1_1 y_0 y_1
End



Now, we convert the instance into a QUBO instance.

In [4]:
qubo_1 = ilp_to_qubo(bp_model_1)

In [5]:
print(qubo_1.lp_string)

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: bin_packing_qubo

Minimize
 obj: 18 x_0_0 + 18 x_0_1 + 144 x_1_0 + 144 x_1_1 + 295 y_0 + 295 y_1 + 96 s_0_0
      + 24 s_0_1 + 6 s_0_2 + 96 s_1_0 + 24 s_1_1 + 6 s_1_2 + [ 24 x_0_0*x_0_1
      + 240 x_0_0*x_1_0 - 336 x_0_0*y_0 + 192 x_0_0*s_0_0 + 96 x_0_0*s_0_1
      + 48 x_0_0*s_0_2 + 240 x_0_1*x_1_1 - 336 x_0_1*y_1 + 192 x_0_1*s_1_0
      + 96 x_0_1*s_1_1 + 48 x_0_1*s_1_2 + 24 x_1_0*x_1_1 - 840 x_1_0*y_0
      + 480 x_1_0*s_0_0 + 240 x_1_0*s_0_1 + 120 x_1_0*s_0_2 - 840 x_1_1*y_1
      + 480 x_1_1*s_1_0 + 240 x_1_1*s_1_1 + 120 x_1_1*s_1_2 - 672 y_0*s_0_0
      - 336 y_0*s_0_1 - 168 y_0*s_0_2 - 672 y_1*s_1_0 - 336 y_1*s_1_1
      - 168 y_1*s_1_2 + 192 s_0_0*s_0_1 + 96 s_0_0*s_0_2 + 48 s_0_1*s_0_2
      + 192 s_1_0*s_1_1 + 96 s_1_0*s_1_2 + 48 s_1_1*s_1_2 ]/2 + 12
Subject To

Bounds
 0 <= x_0_0 <= 1
 0 <= x_0_1 <= 1
 0 <= x_1_0 <= 1
 0 <= x_1_1 <= 1
 0 <= y_0 <= 1
 0 <= y_1 <= 1
 0 <= s_0_0 <= 1
 0 <= s_0_1 <= 

### Solving the QUBO using a bruteforce solver

Now, we solve the QUBO using a bruteforce solve. Since, the bin packing problem is NP-Complete, the bruteforce solver goes over all possible values of the variables and outputs the assignment with the least objective value as the solution.

In [6]:
from brute_solver import *

optimal_value, optimal_solution = brute_force_solver(qubo_1)

In [7]:
print('BRUTEFORCE SOLVER:')
print('')
print('The optimal value of the objective is ', optimal_value)
print('')
print('The optimal solution is the mapping given by \n', optimal_solution)

BRUTEFORCE SOLVER:

The optimal value of the objective is  1

The optimal solution is the mapping given by 
 {'x_0_0': 0, 'x_0_1': 1, 'x_1_0': 0, 'x_1_1': 1, 'y_0': 0, 'y_1': 1, 's_0_0': 0, 's_0_1': 0, 's_0_2': 0, 's_1_0': 0, 's_1_1': 0, 's_1_2': 0}


### Solving the QUBO using DWave simulated annealing

Next, we solve the QUBO using the DWave simulated annealing.

In [13]:
from dwave.samplers import SimulatedAnnealingSampler
from get_qubo_mat import *

sampler = SimulatedAnnealingSampler()

(qubo_matrix, offset) = get_q_matrix_dict(qubo_1)

sampleset = sampler.sample_qubo(qubo_matrix, num_reads=20000)

In [19]:
var_mapping = {}

sample = sampleset.first.sample
for var in sample:
    var_mapping[var] = int(sampleset.first.sample[var])

In [20]:
print('DWAVE ANNEALING SIMULATOR:')
print('')
print('The optimal value of the objective is ', sampleset.first.energy + offset)
print('')
print('The optimal solution is the mapping given by \n', var_mapping)

DWAVE ANNEALING SIMULATOR:

The optimal value of the objective is  1.0

The optimal solution is the mapping given by 
 {'s_0_0': 0, 's_0_1': 0, 's_0_2': 0, 's_1_0': 0, 's_1_1': 0, 's_1_2': 0, 'x_0_0': 0, 'x_0_1': 1, 'x_1_0': 0, 'x_1_1': 1, 'y_0': 0, 'y_1': 1}


### Solving QUBO using Variational Circuits

#### Using sample ansatz 1

In [8]:
from ansatz import *
from vqe_solver import *

num_qubits = qubo_1.number_of_variables
ansatz_1 = create_sample_ansatz1(num_qubits)

min_obj_val_1, final_params_1, final_mapping_dict_1 = solve_qubo_using_vqe(ansatz_1, qubo_1)

In [9]:
print('VARIATIONAL CIRCUITS - ANSATZ 1:')
print('')
print('The optimal value of the objective is ', min_obj_val_1)
print('')
print('The optimal solution is the mapping given by \n', final_mapping_dict_1)

VARIATIONAL CIRCUITS - ANSATZ 1:

The optimal value of the objective is  10.17549970764941

The optimal solution is the mapping given by 
 {'x_0_0': 1, 'x_0_1': 1, 'x_1_0': 0, 'x_1_1': 0, 'y_0': 1, 'y_1': 0, 's_0_0': 0, 's_0_1': 1, 's_0_2': 0, 's_1_0': 0, 's_1_1': 0, 's_1_2': 0}


#### Using sample ansatz 2

In [10]:
ansatz_2 = create_sample_ansatz2(num_qubits)

min_obj_val_2, final_params_2, final_mapping_dict_2 = solve_qubo_using_vqe(ansatz_2, qubo_1)

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [11]:
print('VARIATIONAL CIRCUITS - ANSATZ 2:')
print('')
print('The optimal value of the objective is ', min_obj_val_2)
print('')
print('The optimal solution is the mapping given by \n', final_mapping_dict_2)

VARIATIONAL CIRCUITS - ANSATZ 2:

The optimal value of the objective is  32.25047198402828

The optimal solution is the mapping given by 
 {'x_0_0': 0, 'x_0_1': 0, 'x_1_0': 1, 'x_1_1': 0, 'y_0': 1, 'y_1': 0, 's_0_0': 1, 's_0_1': 0, 's_0_2': 0, 's_1_0': 0, 's_1_1': 1, 's_1_2': 0}


#### Using sample ansatz 3

In [13]:
ansatz_3 = create_twolocal_rx_rz_cx_linear_ansatz(num_qubits, reps=3)

min_obj_val_3, final_params_3, final_mapping_dict_3 = solve_qubo_using_vqe(ansatz_3, qubo_1)

In [14]:
print('VARIATIONAL CIRCUITS - ANSATZ 3:')
print('')
print('The optimal value of the objective is ', min_obj_val_3)
print('')
print('The optimal solution is the mapping given by \n', final_mapping_dict_3)

VARIATIONAL CIRCUITS - ANSATZ 3:

The optimal value of the objective is  19.99244110852262

The optimal solution is the mapping given by 
 {'x_0_0': 0, 'x_0_1': 0, 'x_1_0': 0, 'x_1_1': 1, 'y_0': 0, 'y_1': 1, 's_0_0': 0, 's_0_1': 0, 's_0_2': 1, 's_1_0': 0, 's_1_1': 0, 's_1_2': 0}


#### Using sample ansatz 4

In [16]:
ansatz_4 = create_twolocal_ry_rz_cx_full_ansatz(num_qubits, reps=3)

min_obj_val_4, final_params_4, final_mapping_dict_4 = solve_qubo_using_vqe(ansatz_4, qubo_1)

In [17]:
print('VARIATIONAL CIRCUITS - ANSATZ 4:')
print('')
print('The optimal value of the objective is ', min_obj_val_4)
print('')
print('The optimal solution is the mapping given by \n', final_mapping_dict_4)

VARIATIONAL CIRCUITS - ANSATZ 4:

The optimal value of the objective is  20.335861205568065

The optimal solution is the mapping given by 
 {'x_0_0': 0, 'x_0_1': 0, 'x_1_0': 1, 'x_1_1': 0, 'y_0': 1, 'y_1': 0, 's_0_0': 0, 's_0_1': 0, 's_0_2': 1, 's_1_0': 0, 's_1_1': 0, 's_1_2': 1}


#### Using problem specific ansatz

In [18]:
ansatz_5 = create_problem_specific_ansatz_rx(num_qubits, reps=3)

min_obj_val_5, final_params_5, final_mapping_dict_5 = solve_qubo_using_vqe(ansatz_5, qubo_1)

In [19]:
print('VARIATIONAL CIRCUITS - PROBLEM SPECIFIC ANSATZ:')
print('')
print('The optimal value of the objective is ', min_obj_val_5)
print('')
print('The optimal solution is the mapping given by \n', final_mapping_dict_5)

VARIATIONAL CIRCUITS - PROBLEM SPECIFIC ANSATZ:

The optimal value of the objective is  14.486743964120294

The optimal solution is the mapping given by 
 {'x_0_0': 0, 'x_0_1': 0, 'x_1_0': 0, 'x_1_1': 0, 'y_0': 0, 'y_1': 0, 's_0_0': 0, 's_0_1': 0, 's_0_2': 0, 's_1_0': 0, 's_1_1': 0, 's_1_2': 1}


### Solving the QUBO using QAOA

In [8]:
from qaoa_solver import *

p = 6

min_obj_val_qaoa, _, final_mapping_dict_qaoa = solve_qaoa(qubo_1, p)

In [9]:
print('QAOA SOLVER:')
print('')
print('The optimal value of the objective is ', min_obj_val_qaoa)
print('')
print('The optimal solution is the mapping given by \n', final_mapping_dict_qaoa)

QAOA SOLVER:

The optimal value of the objective is  396.6223137856491

The optimal solution is the mapping given by 
 {'x_0_0': 0, 'x_0_1': 1, 'x_1_0': 0, 'x_1_1': 0, 'y_0': 1, 'y_1': 0, 's_0_0': 0, 's_0_1': 1, 's_0_2': 1, 's_1_0': 1, 's_1_1': 0, 's_1_2': 1}


### Analysis of the results

We can see that the bruteforce solver, as expected, outputs the optimal value and the optimal variable assignment. Similarly, the DWave annealing simulator outputs the expected optimal value with optimal assignment. Since the size of the bin packing problem is small, both the solvers solved the problems in less time.

On the other hand, the solutions due to the VQA are not optimal but close to optimal for almost all of the ansatz. Although the number of optimizations was large (number of optimizations was set to 3000), possibly due to the presense of local minima in the parameter landscape, the models were unable to converge to the optimal solution. Moreover, the variable assigments returned were also not always feasible.

The optimal value returned by the QAOA circuit was far from optimal. This could potentially be due to the small number of intervals (p=6) taken. Because it is well known that the principle of QAOA is quantum annealing and that quantum annealing works better as the number of intervals increase, i.e., when the Hamiltonians are applied slowly. 